# Spike Raster
![status](https://img.shields.io/badge/status-in%20progress-orange)

<div style="text-align: center;">
    <img src="./assets/230524_spike-raster.png" alt="spike raster preview" width="450"/>
</div>


## Summary

The proposed spike raster viewer workflow will handle large-scale neuronal spike time data, offering an efficient scatter-plot-like API with rich interactivity. It will provide linked aggregate views for spike counts over time and channels and manages spike-level metadata, such as motif-group color-coding, for a detailed data presentation.

## Imports and config

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Requirements</p>
    <p>This workflow notebook requires the <a href="./environment.yml">environment</a> specified in this workflow directory.</p>
</div>


In [ ]:
from neurodatagen.ephys import sim_spikes, assign_groups
import holoviews as hv
hv.extension('bokeh')

## Generate spike time data

The `sim_spikes` function simulates spike times for a given number of neurons, each with a specified firing rate, over a particular duration. It calculates the expected number of spikes per neuron using a Poisson distribution, generates the corresponding spike times using a uniform distribution, assigns these times to the neurons, and then returns a pandas DataFrame containing the spike times and neuron IDs, sorted by the spike times.

The function `assign_groups` bins the spike times into a specified number of groups over time, using a probabilistic approach to mimic some amount of overlap in time of the grouping.

In [ ]:
n_neurons = 50
firing_rate = 1 # Hz
duration = 30 # seconds
n_groups = 4

spikes_df = sim_spikes(n_neurons, firing_rate, duration)
spikes_df['group'] = assign_groups(spikes_df.time, n_groups)

In [ ]:
spikes_df.tail()

## Visualize synthetic spike time data

### Version 1: For loop to set `position` of each hv.Spike series for each neuron

In [ ]:
spike_train_opts = {'color':'group',  'cmap':'category10', 
                        'spike_length':.95, 'tools':['hover']}

overlay_opts = {'ylabel':'Neuron', 'xlabel':'Time (s)', 'show_grid':True, 
                         'padding':0.01, 'width':500, 'height':500, 'show_legend':False}
        
# group the DataFrame by the neuron ID col and sort the resulting groups by key
spike_groups = sorted(spikes_df.groupby('neuron'), key=lambda x: x[0])
    
spikes_dict = {}
for ineuron, ispikes in spike_groups:
    spikes_dict[ineuron] = hv.Spikes(ispikes).opts(position=ineuron-.5, **spike_train_opts)

hv.NdOverlay(spikes_dict, kdims=overlay_opts['ylabel']).opts(yticks=spikes_dict.keys, **overlay_opts)
    

### Version 2: hvPlot with rotated dash as marker

- The problem with this implementation is that the markers don't scale when zooming in or out

In [ ]:
import hvplot.pandas

scatter_opts = {'cmap':'category10', 'ylabel':'Neuron', 'xlabel':'Time (s)', 'grid':True, 
                    'size':50, 'tools':['hover'], 'padding':0.01, 'width':500, 'height':500, 
                    'legend':False, 'colorbar':False, 'marker':'dash', 'angle':90, 'color':'group'}

spikes_df.hvplot.scatter('time', 'neuron', **scatter_opts)

## Load and plot real data